# Sesión 3.4 Elasticsearch
Este es un simple ejemplo de uso de Elasticsearch como buscador usando la API de Python de Elasticsearch.

Elasticsearch es un motor de búsqueda y análisis de datos basado en Lucene, diseñado para ofrecer un almacenamiento y búsqueda escalables en tiempo real. Es parte de la pila ELK (Elasticsearch, Logstash y Kibana) y se utiliza ampliamente para aplicaciones que requieren búsquedas de texto completo, análisis de datos en tiempo real y análisis de registros.

Se ejecuta como un servicio independiente y proporciona una API REST que permite a los desarrolladores interactuar con él mediante solicitudes HTTP.

Lo más sencillo para instalar es usar un contenedor Docker sencillo como el que se ha proporcionado en esta práctica mediante el fichero "docker-compose.yml"

Una vez ejecutado el servicio, ya podemos acceder a él. En esta práctica se hará uso del servicio de un servidor del profesor ubicado en "valencia2.inf.um.es"

Para utilizar Elasticsearch desde Python, lo primero que hacemos es instalar las librerías de Elasticsearch

In [ ]:
!pip install elasticsearch

##Apartado 1.0 Resuelto

Con este código obtenemos información del clúster de Elasticsearch

In [ ]:
from elasticsearch import Elasticsearch

# Conexión a Elasticsearch
client = Elasticsearch(['http://valencia2.inf.um.es:9200'])

def obtener_informacion_cluster():
    try:
        # Obtener información sobre el clúster de Elasticsearch
        info_cluster = client.cluster.health()
        print("Información del clúster:")
        print(info_cluster)
    except Exception as e:
        print("Error al obtener información del clúster:", e)

# Llamar a la función para obtener información del clúster
obtener_informacion_cluster()

## Apartado 1.1 Resuelto

Se define un esquema de texto formado por varios campos para representar los documentos. Podemos decir si queremos que se almacene o no.


Después definimos la carpeta donde se va a guardar y crear el índice y vamos añadiendo documentos al índice.

In [ ]:
from elasticsearch import Elasticsearch, helpers
import json

# Configuración de Elasticsearch (agrega verify_certs=False si no es necesario verificar certificados SSL)
es = Elasticsearch(
    [{'host': 'valencia2.inf.um.es', 'port': 9200, 'scheme': 'http'}],
    verify_certs=False
)

# Definición del índice "documents_index" y el mapeo de los campos
index_name = 'documents_index'

# Comprobamos si el índice ya existe y, si es así, lo eliminamos para crearlo de nuevo
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)

# Creamos el índice con la configuración del mapeo
es.indices.create(
    index=index_name,
    body={
        'mappings': {
            'properties': {
                'id': {'type': 'keyword'},
                'path': {'type': 'keyword'},
                'title': {'type': 'text'},
                'text': {'type': 'text'}
            }
        }
    }
)

# Definimos los documentos que queremos añadir al índice
documents = [
    {
        '_index': index_name,
        '_id': '1',
        '_source': {
            'id': '1',
            'path': '1.txt',
            'title': 'Entrega de los primeros dos ejercicios evaluables',
            'text': 'En el aulavirtual están los dos primeros ejercicios evaluables en la sección tareas'
        }
    },
    {
        '_index': index_name,
        '_id': '2',
        '_source': {
            'id': '2',
            'path': '2.txt',
            'title': 'Ya podéis realizar la primera práctica evaluable hasta 3 puntos',
            'text': 'Con la clase práctica hoy de Scrapy ya podéis empezar a realizar la primera práctica evaluable hasta 3 puntos de la asignatura. Se entregarán a través de tareas'
        }
    }
]

# Insertamos los documentos en Elasticsearch
helpers.bulk(es, documents)

print("Documentos indexados correctamente.")



## Apartado 1.2

En el siguiente código se puede hacer una búsqueda sencilla con Elasticsearch.

La documentación de esta librería se encuentra en: https://www.elastic.co/guide/en/elasticsearch/reference/current/search-your-data.html

In [ ]:
from elasticsearch import Elasticsearch

# Configuración de Elasticsearch (asegúrate de que esté en ejecución y accesible)
es = Elasticsearch(
    [{'host': 'valencia2.inf.um.es', 'port': 9200, 'scheme': 'http'}],
    verify_certs=False
)

# Texto que se quiere buscar
texto_a_buscar = 'tareas'

# Definimos el índice donde realizaremos la búsqueda
index_name = 'documents_index'

# Construimos la consulta de búsqueda para buscar en "title" o "text"
query = {
    "query": {
        "multi_match": {
            "query": texto_a_buscar,
            "fields": ["title", "text"]
        }
    },
    "highlight": {
        "fields": {
            "title": {},
            "text": {}
        }
    }
}

# Ejecutamos la búsqueda en Elasticsearch
response = es.search(index=index_name, body=query)

# Procesamos los resultados de la búsqueda
for hit in response['hits']['hits']:
    print("Resultado:")
    print("Score:", hit['_score'])

    # Comprobamos si hay resaltado en "text" o en "title"
    highlighted_text = None
    if 'highlight' in hit:
        if 'text' in hit['highlight']:
            highlighted_text = hit['highlight']['text'][0]
        elif 'title' in hit['highlight']:
            highlighted_text = hit['highlight']['title'][0]

    # Mostramos el texto resaltado o un mensaje indicando que no hay resaltado
    if highlighted_text:
        print("Texto resaltado:", highlighted_text)
    else:
        print("Sin resaltado")


## Apartado 1.3

El índice se podría exportar en un fichero JSON usando un trozo de código parecido al siguiente.

In [ ]:
from elasticsearch import Elasticsearch, helpers
import json

# Configuración de Elasticsearch
es = Elasticsearch(
    [{'host': 'valencia2.inf.um.es', 'port': 9200, 'scheme': 'http'}],
    verify_certs=False
)

# Nombre del índice a exportar
index_name = 'documents_index'

# Abrimos un archivo para guardar los datos
with open('indice_exportado.json', 'w') as file:
    # Usamos el helper scan para obtener todos los documentos del índice
    for doc in helpers.scan(es, index=index_name):
        # Escribimos cada documento en el archivo en formato JSON
        file.write(json.dumps(doc) + '\n')

print("Exportación completada.")


## Apartado 1.4

Con el siguiente código podemos importar de nuevo el índice.

In [ ]:
from elasticsearch import Elasticsearch, helpers
import json

# Configuración de Elasticsearch
es = Elasticsearch(
    [{'host': 'valencia2.inf.um.es', 'port': 9200, 'scheme': 'http'}],
    verify_certs=False
)
# Nombre del nuevo índice a crear
new_index_name = 'documents_index'

# Comprobamos si el índice ya existe y, si es así, lo eliminamos para crearlo de nuevo
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)

# Creamos el nuevo índice (opcionalmente, se puede definir un mapping)
es.indices.create(index=new_index_name)

# Abrimos el archivo que contiene los datos exportados
with open('indice_exportado.json', 'r') as file:
    # Leemos cada línea del archivo y cargamos los documentos en Elasticsearch
    actions = [
        {
            "_index": new_index_name,
            "_id": json.loads(line)['_id'],  # Utilizamos el mismo ID
            "_source": json.loads(line)['_source']
        }
        for line in file
    ]

    # Usamos el helper bulk para cargar los documentos en Elasticsearch
    helpers.bulk(es, actions)

print("Importación completada.")


## Apartado 1.5

Hacemos la misma búsqueda que el apartado 1.3 con este índice cargado.

In [ ]:
from elasticsearch import Elasticsearch

# Configuración de Elasticsearch (asegúrate de que esté en ejecución y accesible)
es = Elasticsearch(
    [{'host': 'valencia2.inf.um.es', 'port': 9200, 'scheme': 'http'}],
    verify_certs=False
)

# Texto que se quiere buscar
texto_a_buscar = 'tareas'

# Definimos el índice donde realizaremos la búsqueda
index_name = 'documents_index'

# Construimos la consulta de búsqueda para buscar en "title" o "text"
query = {
    "query": {
        "multi_match": {
            "query": texto_a_buscar,
            "fields": ["title", "text"]
        }
    },
    "highlight": {
        "fields": {
            "title": {},
            "text": {}
        }
    }
}

# Ejecutamos la búsqueda en Elasticsearch
response = es.search(index=index_name, body=query)

# Procesamos los resultados de la búsqueda
for hit in response['hits']['hits']:
    print("Resultado:")
    print("Score:", hit['_score'])

    # Comprobamos si hay resaltado en "text" o en "title"
    highlighted_text = None
    if 'highlight' in hit:
        if 'text' in hit['highlight']:
            highlighted_text = hit['highlight']['text'][0]
        elif 'title' in hit['highlight']:
            highlighted_text = hit['highlight']['title'][0]

    # Mostramos el texto resaltado o un mensaje indicando que no hay resaltado
    if highlighted_text:
        print("Texto resaltado:", highlighted_text)
    else:
        print("Sin resaltado")